In [1]:
import numpy as np
import xarray as xr

## artificial data

In [2]:
Nt = 73
Nz = 46
Ny = 90
Nx = 180

In [3]:
data = xr.Dataset(
    {
        "T": xr.DataArray(
            np.random.normal(size=(Nt, Nz, Ny, Nx)),
            dims=("t", "z", "y", "x"),
            coords={
                "t": np.arange(Nt),
                "z": np.linspace(0, 5500, Nz),
                "y": np.linspace(-90, 90, Ny),
                "x": np.linspace(-180, 180, Nx),
            },
        ),
        "S": xr.DataArray(
            np.random.normal(size=(Nt, Nz, Ny, Nx)),
            dims=("t", "z", "y", "x"),
            coords={
                "t": np.arange(Nt),
                "z": np.linspace(0, 5500, Nz),
                "y": np.linspace(-90, 90, Ny),
                "x": np.linspace(-180, 180, Nx),
            },
        ),
    }
)

data

<xarray.Dataset>
Dimensions:  (t: 73, z: 46, y: 90, x: 180)
Coordinates:
  * t        (t) int64 0 1 2 3 4 5 6 7 8 9 10 ... 63 64 65 66 67 68 69 70 71 72
  * z        (z) float64 0.0 122.2 244.4 366.7 ... 5.256e+03 5.378e+03 5.5e+03
  * y        (y) float64 -90.0 -87.98 -85.96 -83.93 ... 83.93 85.96 87.98 90.0
  * x        (x) float64 -180.0 -178.0 -176.0 -174.0 ... 174.0 176.0 178.0 180.0
Data variables:
    T        (t, z, y, x) float64 -0.2821 0.5111 -0.8443 ... 0.5396 0.2417
    S        (t, z, y, x) float64 -0.6795 0.5039 1.959 ... -0.2801 -0.8151

## Artificial Argos

In [4]:
Nfloat = 100

In [5]:
argos = xr.Dataset(
    {
        "t": (("float", ), np.random.randint(0, Nt, size=(Nfloat, ))),
        "y": (("float", ), np.random.uniform(-90, 90, size=(Nfloat, ))),
        "x": (("float", ), np.random.uniform(-180, 180, size=(Nfloat, ))),
        "zmax": (("float", ), np.random.uniform(1000, 5500, size=(Nfloat, ))),
    }
)

argos

<xarray.Dataset>
Dimensions:  (float: 100)
Dimensions without coordinates: float
Data variables:
    t        (float) int64 8 9 45 67 14 13 42 22 63 ... 41 19 72 41 55 36 36 4
    y        (float) float64 -9.399 -46.3 -28.24 34.18 ... -58.51 -12.54 -13.07
    x        (float) float64 67.98 -146.3 30.74 46.35 ... -62.27 -83.47 -55.9
    zmax     (float) float64 3.296e+03 3.896e+03 ... 4.372e+03 1.373e+03

## Vectorized selection and masking

In [7]:
data_selected = data.sel(
    t=argos.t,
    y=argos.y,
    x=argos.x,
    method="nearest",
).where(
    data.z < argos.zmax,
    0.0,
)

data_selected

<xarray.Dataset>
Dimensions:  (float: 100, z: 46)
Coordinates:
    t        (float) int64 8 9 45 67 14 13 42 22 63 ... 41 19 72 41 55 36 36 4
  * z        (z) float64 0.0 122.2 244.4 366.7 ... 5.256e+03 5.378e+03 5.5e+03
    y        (float) float64 -9.101 -45.51 -27.3 33.37 ... -57.64 -13.15 -13.15
    x        (float) float64 67.37 -145.8 31.17 47.26 ... -61.34 -83.46 -55.31
Dimensions without coordinates: float
Data variables:
    T        (float, z) float64 -0.0939 0.2793 -1.002 -1.412 ... 0.0 0.0 0.0 0.0
    S        (float, z) float64 -0.8126 0.7078 -0.4357 0.8712 ... 0.0 0.0 0.0